In [1]:
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from imutils import paths
import random
import cv2
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import flwr as fl
# Load and compile Keras model


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)


In [3]:
BATCH_SIZE = 8
train_data_dir = 'D:/TQ/Federated/4/train/'

test_data_dir = 'D:/TQ/Federated/4/val/'


In [4]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Set the validation split percentage
)
test_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)




In [5]:
print("Class Counts in Training Samples:")
total_classes = sorted(os.listdir(train_data_dir))
print(total_classes)
for category in total_classes:
    category_path = os.path.join(train_data_dir, category)
    sample_count = len(os.listdir(category_path))
    print(f"{category}: {sample_count} samples")


Class Counts in Training Samples:
['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
Animal_Abuse: 96 samples
Arson: 128 samples
Fight: 184 samples
Normal: 199 samples
Riot: 220 samples
Traffic_Accident: 240 samples
Train_Accident: 0 samples


In [6]:
train_classes = sorted(os.listdir(train_data_dir))
test_classes = sorted(os.listdir(test_data_dir))


In [7]:
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Freeze the base model layers
base_model.trainable = False

# Add custom classification layers on top of the base model
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Add dropout layer with a dropout rate of 0.5
outputs = Dense(7, activation='softmax')(x)  # Assuming binary classification

# Create the full model
model4 = Model(inputs, outputs)
optimizer = Adam(learning_rate=0.001)
model4.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


In [8]:
traingen = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(225, 225),
                                                   class_mode='categorical',
                                                   classes=total_classes,
                                                   subset='training',
                                                   batch_size=BATCH_SIZE, 
                                                   shuffle=True,
                                                   seed=42)

validgen = train_datagen.flow_from_directory(train_data_dir,
                                               target_size=(225, 225),
                                               class_mode='categorical',
                                               classes=total_classes,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

testgen = test_datagen.flow_from_directory(test_data_dir,
                                             target_size=(225, 225),
                                             class_mode='categorical',
                                             classes=total_classes,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)


Found 856 images belonging to 7 classes.
Found 211 images belonging to 7 classes.
Found 300 images belonging to 7 classes.


In [9]:
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
callbacks = [early_stopping]


In [10]:
history = model4.fit(
            traingen,
            epochs=5,
            steps_per_epoch=len(traingen),
            validation_data=validgen,
            validation_steps=len(validgen),
            callbacks=callbacks,
            verbose = 1
)

Epoch 1/5


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:2616: DeprecationWarning: Please import `affine_transform` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.interpolation.affine_transform(


107/107 [==============================] - 25s 212ms/step - loss: 0.8728 - accuracy: 0.7103 - val_loss: 0.9311 - val_accuracy: 0.7441
Epoch 2/5
107/107 [==============================] - 17s 155ms/step - loss: 0.2241 - accuracy: 0.9334 - val_loss: 0.7869 - val_accuracy: 0.8152
Epoch 3/5
107/107 [==============================] - 17s 157ms/step - loss: 0.1616 - accuracy: 0.9556 - val_loss: 0.8977 - val_accuracy: 0.8104
Epoch 4/5
107/107 [==============================] - 17s 155ms/step - loss: 0.0985 - accuracy: 0.9696 - val_loss: 0.9043 - val_accuracy: 0.8199
Epoch 5/5
107/107 [==============================] - 16s 149ms/step - loss: 0.0856 - accuracy: 0.9731 - val_loss: 0.9755 - val_accuracy: 0.8104


In [11]:
#model = tf.keras.models.load_model("D:/TQ/Federated/Client4_Centralized.h5")

In [12]:
y_true = testgen.classes

# Calculate predictions on the test data
y_pred = model4.predict(testgen)
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the predicted class labels

300/300 [==============================] - 8s 23ms/step


In [13]:
print("Accuracy:")
test_loss, test_accuracy = model4.evaluate(testgen, steps=len(testgen), verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Accuracy:
Test Loss: 0.8454536199569702
Test Accuracy: 0.8966666460037231


In [14]:
# Calculate the confusion matrix
cm2 = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix Before Aggregation:")
print(cm2)

Confusion Matrix Before Aggregation:
[[24  0  0  0  0  0  0]
 [ 0 32  0  0  0  0  0]
 [ 0  0 46  0  0  0  0]
 [ 0  0  0 44  0  6  0]
 [ 0  0  0  0 65  0  0]
 [ 0  0  0  1  0 59  0]
 [ 0  0  0  8  0 15  0]]


In [53]:
from sklearn.metrics import confusion_matrix
import numpy as np


# We will store the results in a dictionary for easy access later
per_class_accuracies = {}

# Calculate the accuracy for each one of our classes
for idx, cls in enumerate(total_classes):
    true_positives = cm2[idx, idx]
    
    # Total instances of the current class
    total_instances = np.sum(cm2[idx, :])
    
    # The accuracy for the current class is the ratio between correct predictions to all predictions
    per_class_accuracies[cls] = true_positives / total_instances if total_instances > 0 else 0
    print(f"{cls} - True Positives: {true_positives}, Total Instances: {total_instances}")

print(per_class_accuracies)

Animal_Abuse - True Positives: 24, Total Instances: 24
Arson - True Positives: 32, Total Instances: 32
Fight - True Positives: 46, Total Instances: 46
Normal - True Positives: 44, Total Instances: 50
Riot - True Positives: 65, Total Instances: 65
Traffic_Accident - True Positives: 59, Total Instances: 60
Train_Accident - True Positives: 0, Total Instances: 23
{'Animal_Abuse': 1.0, 'Arson': 1.0, 'Fight': 1.0, 'Normal': 0.88, 'Riot': 1.0, 'Traffic_Accident': 0.9833333333333333, 'Train_Accident': 0.0}


In [15]:
import pandas as pd
from openpyxl import Workbook

categories = ['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
df_cm = pd.DataFrame(cm2, index=categories, columns=categories)

# Write the confusion matrix DataFrame to an Excel file
excel_writer = pd.ExcelWriter('D:/TQ/Federated/confusion_matrix.xlsx')
# Create an Excel workbook and write the confusion matrix DataFrame to a sheet
wb = Workbook()
ws = wb.active
ws.title = 'Confusion Matrix'

# Write the column names
ws.append([''] + df_cm.columns.tolist())

# Write the confusion matrix data
for index, row in df_cm.iterrows():
    ws.append([index] + row.tolist())

# Save the workbook to an Excel file
wb.save('D:/TQ/Federated/confusion_matrix4.xlsx')

print("Confusion matrix written to confusion_matrix.xlsx")

Confusion matrix written to confusion_matrix.xlsx


In [16]:
model4.save("Client4_Centralized.h5")

In [17]:
from sklearn.metrics import classification_report 
print(classification_report(y_true, y_pred_labels, target_names=total_classes))


                  precision    recall  f1-score   support

    Animal_Abuse       1.00      1.00      1.00        24
           Arson       1.00      1.00      1.00        32
           Fight       1.00      1.00      1.00        46
          Normal       0.83      0.88      0.85        50
            Riot       1.00      1.00      1.00        65
Traffic_Accident       0.74      0.98      0.84        60
  Train_Accident       0.00      0.00      0.00        23

        accuracy                           0.90       300
       macro avg       0.80      0.84      0.81       300
    weighted avg       0.84      0.90      0.87       300



C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [49]:
client4 = tf.keras.models.load_model('client4_30_fedavg.h5')

C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


In [50]:
y_true = testgen.classes

# Calculate predictions on the test data
y_pred = client4.predict(testgen)
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the predicted class labels
test_loss, test_accuracy = client4.evaluate(testgen, steps=len(testgen), verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

print(classification_report(y_true, y_pred_labels, target_names=total_classes))


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:2616: DeprecationWarning: Please import `affine_transform` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.interpolation.affine_transform(


300/300 [==============================] - 4s 13ms/step
Test Loss: 0.30904462933540344
Test Accuracy: 0.9100000262260437
                  precision    recall  f1-score   support

    Animal_Abuse       1.00      0.96      0.98        24
           Arson       1.00      1.00      1.00        32
           Fight       0.98      0.98      0.98        46
          Normal       0.69      0.90      0.78        50
            Riot       1.00      0.98      0.99        65
Traffic_Accident       0.92      0.98      0.95        60
  Train_Accident       1.00      0.26      0.41        23

        accuracy                           0.91       300
       macro avg       0.94      0.87      0.87       300
    weighted avg       0.93      0.91      0.90       300



In [51]:
cm = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix Before Aggregation:")
print(cm)

Confusion Matrix Before Aggregation:
[[23  0  1  0  0  0  0]
 [ 0 32  0  0  0  0  0]
 [ 0  0 45  1  0  0  0]
 [ 0  0  0 45  0  5  0]
 [ 0  0  0  1 64  0  0]
 [ 0  0  0  1  0 59  0]
 [ 0  0  0 17  0  0  6]]


In [52]:
import pandas as pd
from openpyxl import Workbook

categories = ['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
df_cm = pd.DataFrame(cm, index=categories, columns=categories)

# Write the confusion matrix DataFrame to an Excel file
excel_writer = pd.ExcelWriter('D:/TQ/Federated/confusion_matrix.xlsx')
# Create an Excel workbook and write the confusion matrix DataFrame to a sheet
wb = Workbook()
ws = wb.active
ws.title = 'Confusion Matrix'

# Write the column names
ws.append([''] + df_cm.columns.tolist())

# Write the confusion matrix data
for index, row in df_cm.iterrows():
    ws.append([index] + row.tolist())

# Save the workbook to an Excel file
wb.save('D:/TQ/Federated/confusion_matrix4f.xlsx')

print("Confusion matrix written to confusion_matrix.xlsx")

Confusion matrix written to confusion_matrix.xlsx


In [48]:
from sklearn.metrics import confusion_matrix
import numpy as np


# We will store the results in a dictionary for easy access later
per_class_accuracies = {}

# Calculate the accuracy for each one of our classes
for idx, cls in enumerate(total_classes):
    true_positives = cm[idx, idx]
    
    # Total instances of the current class
    total_instances = np.sum(cm[idx, :])
    
    # The accuracy for the current class is the ratio between correct predictions to all predictions
    per_class_accuracies[cls] = true_positives / total_instances if total_instances > 0 else 0
    print(f"{cls} - True Positives: {true_positives}, Total Instances: {total_instances}")

print(per_class_accuracies)

Animal_Abuse - True Positives: 22, Total Instances: 24
Arson - True Positives: 31, Total Instances: 32
Fight - True Positives: 39, Total Instances: 46
Normal - True Positives: 46, Total Instances: 50
Riot - True Positives: 65, Total Instances: 65
Traffic_Accident - True Positives: 60, Total Instances: 60
Train_Accident - True Positives: 4, Total Instances: 23
{'Animal_Abuse': 0.9166666666666666, 'Arson': 0.96875, 'Fight': 0.8478260869565217, 'Normal': 0.92, 'Riot': 1.0, 'Traffic_Accident': 1.0, 'Train_Accident': 0.17391304347826086}
